In [1]:
import pandas as pd
import time
import numpy as np
import os
import pandas_gbq
from google.cloud import bigquery
import basedosdados as bd

In [2]:
query = "SELECT  ano,  tipo_vinculo,  sigla_uf,  sum(quantidade_vinculo) as quantidade_vinculos FROM  `basedosdados.br_ibge_estadic.indicadores_quantidade_vinculo` where ano =2021 group by   1, 2, 3"

In [3]:
client = bigquery.Client()

In [4]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')

Downloading: 100%|██████████| 135/135 [00:00<00:00, 308.21rows/s]


In [5]:
df

,ano,tipo_vinculo,sigla_uf,quantidade_vinculos
0,2021,Estatutários,RO,35789
1,2021,Estatutários,AC,22278
2,2021,Estatutários,AM,64630
3,2021,Estatutários,RR,14399
4,2021,Estatutários,PA,81810
...,...,...,...,...
130,2021,Sem vínculo permanente,RS,32044
131,2021,Sem vínculo permanente,MS,10465
132,2021,Sem vínculo permanente,MT,15806
133,2021,Sem vínculo permanente,GO,18283


In [7]:
y=df.pivot_table(index={'sigla_uf'},values='quantidade_vinculos', aggfunc=np.sum)

In [8]:
y = pd.DataFrame(y)
total = y.reset_index()


In [9]:
total=total.rename(columns={'quantidade_vinculos':'total_estado'})

In [10]:
df1 = df.merge(total, right_on='sigla_uf', left_on='sigla_uf')

In [18]:
df1[df1['sigla_uf']=="ES"]

,ano,tipo_vinculo,sigla_uf,quantidade_vinculos,total_estado
85,2021,Estatutários,ES,26830,52807
86,2021,Celetistas,ES,235,52807
87,2021,Somente comissionados,ES,2934,52807
88,2021,Estagiários,ES,1090,52807
89,2021,Sem vínculo permanente,ES,21718,52807


In [14]:
## Subindo para GBQ

dataset_ref = client.dataset('perfil_remuneracao') 

In [29]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135 entries, 0 to 134
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ano                  135 non-null    Int64 
 1   tipo_vinculo         135 non-null    object
 2   sigla_uf             135 non-null    object
 3   quantidade_vinculos  132 non-null    Int64 
 4   total_estado         135 non-null    Int64 
dtypes: Int64(3), object(2)
memory usage: 6.7+ KB


In [15]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referencia da informacao'),
bigquery.SchemaField('tipo_vinculo','STRING',description='Tipo de vinculo.'),
 bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação.'),
 bigquery.SchemaField('quantidade_vinculos','INTEGER',description='Quantidade de vinculos'),
 bigquery.SchemaField('total_estado','INTEGER',description='Total de vínculos daquela UF')
 ]

In [16]:
table_ref = dataset_ref.table('ESTADIC_quantidade_vinculos_mapa') # nome da tabela no padrão FONTE_algo_intuitivo_dado
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df1, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=7d82a612-996f-4c28-b214-8a50d3c71741>